In [1]:
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import streamlit as st
from api_caller import api_call
from helper_clean import historical_data_cleaner 
import numpy as np
from MCForecastTools import MCSimulation

In [33]:
# Read in the csv file
testing_csv = pd.read_csv('project_1_mockdata.csv')

# Create a list of the tickers
tickers = testing_csv['Ticker'].to_list()

alpaca = api_call()

timeframe = "1Day"

# Create a start and end variables to pull data during this period
end = pd.Timestamp("2020-07-13", tz="America/New_York").isoformat()
start = pd.Timestamp("2020-07-05", tz="America/New_York").isoformat()

# Pull the stock data for our tickers and selected times
df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = start,
    end = end
).df

df_portfolio

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2020-07-06 04:00:00+00:00,2934.97,3059.8800,2930.0000,3057.04,6880652,253596,3008.206281,AMZN
2020-07-07 04:00:00+00:00,3058.01,3069.5500,2990.0000,3000.12,5257687,224410,3034.469771,AMZN
2020-07-08 04:00:00+00:00,3021.55,3083.9700,3012.4301,3081.11,5037784,188712,3053.037063,AMZN
2020-07-09 04:00:00+00:00,3116.00,3193.8799,3074.0016,3182.63,6388661,242435,3134.994624,AMZN
2020-07-10 04:00:00+00:00,3191.76,3215.0000,3135.7000,3200.00,5485988,212670,3185.414623,AMZN
2020-07-13 04:00:00+00:00,3251.06,3344.2900,3068.3901,3104.00,7720550,352980,3235.104744,AMZN
2020-07-06 04:00:00+00:00,1480.06,1506.5900,1472.8600,1495.70,1563997,56188,1492.376734,GOOG
2020-07-07 04:00:00+00:00,1490.00,1516.8000,1483.5500,1485.18,1458216,49011,1500.388846,GOOG
2020-07-08 04:00:00+00:00,1494.32,1505.8799,1485.6300,1496.00,1249687,41008,1496.021181,GOOG


In [34]:
AMZN_ = df_portfolio[df_portfolio['symbol']=='AMZN'].drop('symbol', axis=1)
GOOG_ = df_portfolio[df_portfolio['symbol']=='GOOG'].drop('symbol', axis=1)
JPM_ = df_portfolio[df_portfolio['symbol']=='JPM'].drop('symbol', axis=1)
WMT_ = df_portfolio[df_portfolio['symbol']=='WMT'].drop('symbol', axis=1)

simulation_df = pd.concat([AMZN_, GOOG_, JPM_, WMT_], axis=1, keys=['AMZN', 'GOOG', 'JPM', 'WMT'])

simulation_df

AMZN                                          \
                              open       high        low    close   volume   
timestamp                                                                    
2020-07-06 04:00:00+00:00  2934.97  3059.8800  2930.0000  3057.04  6880652   
2020-07-07 04:00:00+00:00  3058.01  3069.5500  2990.0000  3000.12  5257687   
2020-07-08 04:00:00+00:00  3021.55  3083.9700  3012.4301  3081.11  5037784   
2020-07-09 04:00:00+00:00  3116.00  3193.8799  3074.0016  3182.63  6388661   
2020-07-10 04:00:00+00:00  3191.76  3215.0000  3135.7000  3200.00  5485988   
2020-07-13 04:00:00+00:00  3251.06  3344.2900  3068.3901  3104.00  7720550   

                                                       GOOG             \
                          trade_count         vwap     open       high   
timestamp                                                                
2020-07-06 04:00:00+00:00      253596  3008.206281  1480.06  1506.5900   
2020-07-07 04:00:00+00:00      224410  3034.469771  1490.00  1516.8000   
2020-07-08 04:00:00+00:00      188712  3053.037063  1494.32  1505.8799   
2020-07-09 04:00:00+00:00      242435  3134.994624  1506.45  1522.7200   
2020-07-10 04:00:00+00:00      212670  3185.414623  1506.15  1543.8300   
2020-07-13 04:00:00+00:00      352980  3235.104744  1551.12  1577.1316   

                                      ...       JPM                         \
                                 low  ...    volume trade_count       vwap   
timestamp                             ...                                    
2020-07-06 04:00:00+00:00  1472.8600  ...  18625431      127956  94.428021   
2020-07-07 04:00:00+00:00  1483.5500  ...  25757803      147216  92.623059   
2020-07-08 04:00:00+00:00  1485.6300  ...  18762297      126424  92.635983   
2020-07-09 04:00:00+00:00  1488.0850  ...  25600919      187622  91.663501   
2020-07-10 04:00:00+00:00  1496.5400  ...  28954234      213530  94.924184   
2020-07-13 04:00:00+00:00  1505.2427  ...  32079211      242070  97.764589   

                              WMT                                    \
                             open    high     low   close    volume   
timestamp                                                             
2020-07-06 04:00:00+00:00  119.80  119.87  118.22  118.89   7231664   
2020-07-07 04:00:00+00:00  118.45  127.55  118.22  126.95  31152676   
2020-07-08 04:00:00+00:00  128.00  128.13  124.40  124.44  17405804   
2020-07-09 04:00:00+00:00  125.61  128.18  124.95  127.75  14257238   
2020-07-10 04:00:00+00:00  128.71  131.37  127.26  130.68  14752333   
2020-07-13 04:00:00+00:00  131.33  133.63  128.89  129.52  14112829   

                                                   
                          trade_count        vwap  
timestamp                                          
2020-07-06 04:00:00+00:00       73823  118.862227  
2020-07-07 04:00:00+00:00      240017  124.617704  
2020-07-08 04:00:00+00:00      168093  125.860774  
2020-07-09 04:00:00+00:00      126303  126.923580  
2020-07-10 04:00:00+00:00      125936  129.564404  
2020-07-13 04:00:00+00:00      144274  131.254985  

[6 rows x 28 columns]